# Grab PlumX Data

## Request Functions
This part contains functions we need to fetch the web data and should also handle the exceptions while fetching here.

In [1]:
from grab_util import *
import json
import pandas as pd
import numpy as np
import math
import gc


In [38]:
origin_folder = '/Volumes/Seagate Backup Plus Drive/Dev/altmetric/Data_B'

## Grab PlumX from API

In [3]:
all_articles_df = pd.read_excel(f'{origin_folder}/all.xlsx', usecols=['SO', 'DI'])
# all_articles_df = pd.read_csv(f'{origin_folder}/all.csv', usecols=['SO', 'DI'])

In [4]:
plumx_info = {}

In [5]:
for index, doi in enumerate(all_articles_df['DI']):
    if plumx_info.__contains__(doi):
        continue
    if index % 100 == 0:
        print(index)
    res = grab_from_url_json('https://plu.mx/api/v1/artifact/doi/' + str(doi), timeout=60)
    plumx_info[doi] = res

len(plumx_info)

SConnection object at 0x12713e208>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
ConnectionError: HTTPSConnectionPool(host='plu.mx', port=443): Max retries exceeded with url: /api/v1/artifact/doi/10.1021/jacs.7b04821 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x12713e7b8>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
ConnectionError: HTTPSConnectionPool(host='plu.mx', port=443): Max retries exceeded with url: /api/v1/artifact/doi/10.1021/jacs.7b04821 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x12714e0b8>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
ConnectionError: HTTPSConnectionPool(host='plu.mx', port=443): Max retries exceeded with url: /api/v1/artifact/doi/10.1021/jacs.7b04821 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnec

KeyboardInterrupt: 

In [13]:
with open(f'{origin_folder}/article_plumx.json', "w+") as dump_f:
    dump_f.write(json.dumps(plumx_info))

## Generate Plumx Ids

In [4]:
with open(f'{origin_folder}/article_plumx.json', "r") as f:
    plumx_info = json.load(f)


In [ ]:
df = pd.DataFrame(columns=['DI', 'plumx_id'])
for doi, item in plumx_info.items():
    if doi == '' or str(doi).lower() == 'nan' or 'id' not in item.keys():
        continue
    dfx = pd.DataFrame(columns=['DI', 'plumx_id'])
    dfx['DI'] = [doi]
    dfx['plumx_id'] = [item['id']]
    df = df.append(dfx, ignore_index=True)

In [ ]:
df.to_csv(f'{origin_folder}/article_plumx_id.csv', index=False)

## Grab Plumx Tweets Info

In [ ]:
df = pd.read_csv(f'{origin_folder}/article_plumx_id.csv')
plumx_tweets_info = {}

In [ ]:
%%time
for index, row in df.iterrows():
    if plumx_tweets_info.__contains__(row['DI']):
        continue
    if index % 100 == 0:
        print(index)
    res = grab_from_url_json(f'https://plu.mx/api/v1/artifact/id/{row["plumx_id"]}/metricsEvidence/twitter?offset=0&size=100000')
    plumx_tweets_info[row['DI']] = res


In [ ]:
with open(f'{origin_folder}/article_tweets_plumx.json', "w+") as dump_f:
    dump_f.write(json.dumps(plumx_tweets_info))

In [ ]:
del plumx_tweets_info
gc.collect()

In [ ]:
article_tweets = pd.read_json(f'{origin_folder}/article_tweets_plumx.json', "r")

df_tweets = pd.DataFrame(columns=['doi', 'ref', 'tweet_id', 'tweet_url'])

In [ ]:
%%time
for doi in article_tweets:
    if type(article_tweets[doi]['twitter']) == float:
        continue
    for tweet in article_tweets[doi]['twitter']['artifacts']:
        if tweet['id'] in df_tweets['tweet_id']:
            continue
        df_tweet = pd.DataFrame(columns=['doi', 'ref', 'tweet_id', 'tweet_url'])
        df_tweet['doi'] = [doi]
        df_tweet['ref'] = [tweet['ref_id']['value']]
        df_tweet['tweet_id'] = [tweet['id']]
        df_tweet['tweet_url'] = [tweet['title']['link']]

        df_tweets = df_tweets.append(df_tweet, ignore_index=True)

In [ ]:
df_tweets.to_csv(f'{origin_folder}/article_tweets_plumx.csv', index=False)

## Grab Plumx Details

In [ ]:
def grab_detail_plumx(df, doi_column):
    # USAGE
    abstruct_views = list()
    full_text_views = list()
    link_click_count = list()
    link_outs = list()

    # CAPTURE
    exports_saves = list()
    reader_count_mendeley = list()
    reader_count_citeulike = list()

    # CITATION
    cited_by_count_scopus = list()
    cited_by_count_crossref = list()
    cited_by_count_pubmed = list()

    # SOCIAL MEDIA
    twitter = list()
    facebook = list()

    # MENTION
    news = list()
    blogs = list()
    reference_count_wikipedia = list()
    comment_count_reddit = list()
    mention_qa_site_mentions = list()

    a_list = [abstruct_views, full_text_views, link_click_count, link_outs, exports_saves,
              reader_count_mendeley, reader_count_citeulike, cited_by_count_scopus,
              cited_by_count_crossref, cited_by_count_pubmed, twitter, facebook, news, blogs,
              reference_count_wikipedia, comment_count_reddit, mention_qa_site_mentions]
    b_list = ['abstruct_views', 'full_text_views', 'link_click_count', 'link_outs', 'exports_saves',
              'reader_count_mendeley', 'reader_count_citeulike', 'cited_by_count_scopus',
              'cited_by_count_crossref', 'cited_by_count_pubmed', 'tweets', 'facebook', 'news', 'blogs',
              'reference_count_wikipedia', 'comment_count_reddit', 'mention_qa_site_mentions']

    for index, doi in enumerate(df[doi_column]):
        if index % 100 == 0:
            print(index)

        abstruct_views_count, full_text_views_count, link_click_count_count, link_outs_count, \
        exports_saves_count, reader_count_mendeley_count, reader_count_citeulike_count, \
        cited_by_count_scopus_count, cited_by_count_crossref_count, \
        cited_by_count_pubmed_count, twitter_count, facebook_count, news_count, blogs_count, \
        reference_count_wikipedia_count, comment_count_reddit_count, mention_qa_site_mentions_count \
            = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

        # res = grab_from_url_json('https://plu.mx/api/v1/artifact/doi/' + str(doi))
        res = plumx_info.get(str(doi))
        if res is not None:
            if 'sort_count' in res:
                sort_count = res['sort_count']

                # USAGE
                if 'usage' in sort_count:
                    counts = sort_count['usage']['count_types']
                    for item in counts:
                        if item['name'] == 'ABSTRACT_VIEWS':
                            abstruct_views_count = item['total']
                        elif item['name'] == 'FULL_TEXT_VIEWS':
                            full_text_views_count = item['total']
                        elif item['name'] == 'LINK_CLICK_COUNT':
                            link_click_count_count = item['total']
                        elif item['name'] == 'LINK_OUTS':
                            link_outs_count = item['total']

                # CAPTURE
                if 'capture' in sort_count:
                    counts = sort_count['capture']['count_types']
                    for item in counts:
                        if item['name'] == 'EXPORTS_SAVES':
                            exports_saves_count += item['total']
                        elif item['name'] == 'READER_COUNT':
                            for source in item['sources']:
                                if source['name'] == 'Mendeley':
                                    reader_count_mendeley_count += source['total']
                                elif source['name'] == 'CiteULike':
                                    reader_count_citeulike_count += source['total']

                # CITATION
                if 'citation' in sort_count:
                    counts = sort_count['citation']['count_types']
                    for item in counts:
                        if item['name'] == 'Scopus':
                            cited_by_count_scopus_count = item['total']
                        elif item['name'] == 'CrossRef':
                            cited_by_count_crossref_count = item['total']
                        elif item['name'] == 'PubMed':
                            cited_by_count_pubmed_count = item['total']

                # SOCIAL MEDIA
                if 'social_media' in sort_count:
                    counts = sort_count['social_media']['count_types']
                    for item in counts:
                        if item['name'] == 'TWEET_COUNT':
                            twitter_count = item['total']
                        elif item['name'] == 'FACEBOOK_COUNT':
                            facebook_count = item['total']

                # MENTION
                if 'mention' in sort_count:
                    counts = sort_count['mention']['count_types']
                    for item in counts:
                        if item['name'] == 'NEWS_COUNT':
                            news_count = item['total']
                        elif item['name'] == 'ALL_BLOG_COUNT':
                            blogs_count = item['total']
                        elif item['name'] == 'REFERENCE_COUNT':
                            for source in item['sources']:
                                if source['name'] == 'Wikipedia':
                                    reference_count_wikipedia_count += source['total']
                        elif item['name'] == 'COMMENT_COUNT':
                            for source in item['sources']:
                                if source['name'] == 'Reddit':
                                    comment_count_reddit_count += source['total']
                        elif item['name'] == 'QA_SITE_MENTIONS':
                            mention_qa_site_mentions_count = item['total']

        c_list = [abstruct_views_count, full_text_views_count, link_click_count_count, link_outs_count,
                  exports_saves_count, reader_count_mendeley_count, reader_count_citeulike_count,
                  cited_by_count_scopus_count, cited_by_count_crossref_count,
                  cited_by_count_pubmed_count, twitter_count, facebook_count, news_count, blogs_count,
                  reference_count_wikipedia_count, comment_count_reddit_count,
                  mention_qa_site_mentions_count]

        for i in range(0, len(a_list)):
            a_list[i].append(c_list[i])

    for i in range(0, len(a_list)):
        print(len(a_list[i]))
        df[b_list[i]] = a_list[i]

    return df


In [ ]:
# df = pd.read_csv(f'{origin_folder}/all.csv', usecols=['SO', 'DI'])
# df = pd.read_csv(f'{origin_folder}/article_tweets_altmetric.csv', usecols=['doi'])
# df = df.drop_duplicates()
# print(len(df))
df = grab_detail_plumx(all_articles_df, 'DI')
df.to_csv(f'{origin_folder}/all_plumx_detail.csv', index=False)

In [ ]:
len(df)

In [39]:
df_all = pd.read_excel(f'{origin_folder}/all.xlsx')
df_alt = pd.read_csv(f'{origin_folder}/all_altmetric_detail.csv')
df_plu = pd.read_csv(f'{origin_folder}/all_plumx_detail.csv')
df_plu_id = pd.read_csv(f'{origin_folder}/article_plumx_id.csv')

df_plu = df_plu.drop('SO', axis=1)
df_plu = pd.merge(df_plu, df_plu_id, on='DI', how='left')
df_plu[df_plu['plumx_id'].isnull()] = df_plu[df_plu['plumx_id'].isnull()].replace(0, np.NaN)

df_alt = df_alt.replace(-1, np.NaN)
df_alt[df_alt['altmetric_id'].isnull()] = df_alt[df_alt['altmetric_id'].isnull()].replace(0, np.NaN)

df_alt = df_alt.rename(columns=(lambda x:f'alt_{x}' if x != 'DI' else x))
df_plu = df_plu.rename(columns=(lambda x:f'plu_{x}' if x != 'DI' else x))

df_all = pd.merge(df_all, df_alt, on='DI', how='left')
df_all = df_all = pd.merge(df_all, df_plu, on='DI', how='left')
df_all = df_all.drop_duplicates()

In [40]:
df_all.to_excel(f'{origin_folder}/Data_x.xlsx', index=False)